## [146. LRU Cache](https://leetcode.com/problems/lru-cache/)

问题难度: &clubs; &clubs; &clubs; &clubs; &clubs;

### 问题描述

设计和实现一个`LRU`(`Least Recently Used`)缓存的数据结构. 它应该支持以下的操作: `get`和`put`.

- `get(key)`: 如果该`key`存储于缓存中, 获得该键的值(一般为正数), 否则的话返回`-1`
- `put(key)`: 如果该`key`不存在, 设置或者插入该值. 当缓存到达容量极点时, 应该在插入新条目之前使得最近未使用的条目


**follow up**:
能够在`O(1)`时间复杂度内实现上述的操作.

**示例**
```
LRUCache cache = new LRUCache( 2 /* capacity */ );

cache.put(1, 1);
cache.put(2, 2);
cache.get(1);       // returns 1
cache.put(3, 3);    // evicts key 2
cache.get(2);       // returns -1 (not found)
cache.put(4, 4);    // evicts key 1
cache.get(1);       // returns -1 (not found)
cache.get(3);       // returns 3
cache.get(4);       // returns 4
```

### 解题思路

看到`O(1)`复杂度, 那么直接想到使用一个`hash`结构(data_dict)来存储`key-value`的值, 这样就能够在`O(1)`时间复杂度内实现`get(key)`了...
但是考虑到`put`时,需要考虑缓存到容量极点时,也需要在`O(1)`时间复杂度内完成, 因此需要使用一个类似于栈的结构来存储`key`最近的访问顺序(`visit_list`), `put`的具体逻辑如下:

- 如果`key`已经存在, `data_dict[key] = value`, `visit_list.remove(key), visit_list.append(key)`
- 如果`key`不存在, `least_used = visit_list[0], visit_list = visit_list[1:], visit_list.append(key), data_dict.del(least_used)`, 接着: `data_dict[key] = value, visit_list.append(key)`

然后`remove`的时间复杂度为`O(n)`, 显然还是可以改进的.


使用一个`hash`结构和一个双向链表进行求解, 这样删除节点的时候就可以使得时间复杂度降为`O(1)`了, 新建一个数据结构`Node`, 用来记录前驱节点和后序节点.

### 代码


In [2]:
class LRUCache(object):
    
    capacity = 0
    data_dict = {}
    visit_list = []
    

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity
        self.data_dict = dict()
        self.visit_list = list()
        

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key in self.data_dict:
            self.visit_list.remove(key)
            self.visit_list.append(key)
            return self.data_dict[key]
        else:
            return -1
        
    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        if key in self.data_dict:
            self.visit_list.remove(key)
            self.visit_list.append(key)
            self.data_dict[key] = value
        else:
            if len(self.data_dict) == self.capacity:
                least_used = self.visit_list[0]
                del self.data_dict[least_used]
                self.visit_list = self.visit_list[1:]
            self.data_dict[key] = value
            self.visit_list.append(key)
        
# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [4]:
class Node(object):
    
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.pre = None
        self.next = None
    
class LRUCache(object):
    
    capacity = 0
    node_dict = {}
    head = None
    tail = None
    

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.capacity = capacity
        self.node_dict = dict()
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.pre = self.head
        

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key in self.node_dict:
            node = self.node_dict[key]
            self._remove(node)
            self._add(node)
            return node.value
        else:
            return -1
        
    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        node = Node(key, value)
        if key in self.node_dict:
            pre_node = self.node_dict[key]
            self._remove(pre_node)
        else:
            if len(self.node_dict) == self.capacity:
                n = self.head.next
                self._remove(n)
                del self.node_dict[n.key]
        self.node_dict[key] = node
        self._add(node)
    
    def _add(self, node):
        p = self.tail.pre
        p.next = node
        node.pre = p
        node.next = self.tail
        self.tail.pre = node
    
    def _remove(self, node):
        p = node.pre
        n = node.next
        p.next = n
        n.pre = p
        
# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)